In [1]:
!pip install tensorflow==2.0.0
import tensorflow as tf
import numpy as np
import pandas as pd


In [8]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [9]:
cd drive/Shared\ drives/'Hackathon - ShouldICheckThis'/


/content/drive/Shared drives/Hackathon - ShouldICheckThis


In [7]:
ls

sample_data/


#Data Inspection

In [0]:
meta_data = pd.read_csv('Data/skin-cancer-mnist-ham10000/HAM10000_metadata.csv')

In [0]:
meta_data.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear


In [11]:
meta_data['localization'].value_counts()

back               2192
lower extremity    2077
trunk              1404
upper extremity    1118
abdomen            1022
face                745
chest               407
foot                319
unknown             234
neck                168
scalp               128
hand                 90
ear                  56
genital              48
acral                 7
Name: localization, dtype: int64

In [0]:
meta_data['sex'].value_counts()

male       5406
female     4552
unknown      57
Name: sex, dtype: int64

In [0]:
check = pd.read_csv('Data/skin-cancer-mnist-ham10000/hmnist_28_28_RGB.csv')

In [0]:
check.head()

,pixel0000,pixel0001,pixel0002,pixel0003,pixel0004,pixel0005,pixel0006,pixel0007,pixel0008,pixel0009,pixel0010,pixel0011,pixel0012,pixel0013,pixel0014,pixel0015,pixel0016,pixel0017,pixel0018,pixel0019,pixel0020,pixel0021,pixel0022,pixel0023,pixel0024,pixel0025,pixel0026,pixel0027,pixel0028,pixel0029,pixel0030,pixel0031,pixel0032,pixel0033,pixel0034,pixel0035,pixel0036,pixel0037,pixel0038,pixel0039,...,pixel2313,pixel2314,pixel2315,pixel2316,pixel2317,pixel2318,pixel2319,pixel2320,pixel2321,pixel2322,pixel2323,pixel2324,pixel2325,pixel2326,pixel2327,pixel2328,pixel2329,pixel2330,pixel2331,pixel2332,pixel2333,pixel2334,pixel2335,pixel2336,pixel2337,pixel2338,pixel2339,pixel2340,pixel2341,pixel2342,pixel2343,pixel2344,pixel2345,pixel2346,pixel2347,pixel2348,pixel2349,pixel2350,pixel2351,label
0,192,153,193,195,155,192,197,154,185,202,162,192,208,165,201,209,165,205,208,165,194,212,169,198,216,181,204,216,178,204,215,172,205,217,173,209,217,166,202,214,...,216,181,212,215,176,208,213,168,191,209,159,184,212,168,194,212,172,198,208,173,193,203,168,183,190,142,152,177,123,134,173,124,138,183,147,166,185,154,177,2
1,25,14,30,68,48,75,123,93,126,158,128,158,172,136,162,175,134,157,177,126,158,186,138,163,197,153,187,198,153,186,203,168,206,205,169,204,207,164,199,205,...,209,176,203,207,171,191,208,172,197,205,169,188,198,159,172,200,159,192,191,140,182,177,128,141,147,101,97,113,79,82,60,39,55,25,14,28,25,14,27,2
2,192,138,153,200,145,163,201,142,160,206,149,165,207,149,159,209,156,164,202,136,130,200,133,129,208,146,157,212,159,167,214,157,168,211,143,150,215,148,169,213,...,221,168,175,222,170,188,221,167,184,203,144,155,201,146,155,210,163,177,190,142,152,192,152,172,194,153,176,178,134,149,167,129,143,159,124,142,136,104,117,2
3,38,19,30,95,59,72,143,103,119,171,125,134,177,118,123,190,137,146,192,127,143,200,141,162,207,156,170,202,144,151,202,138,155,212,159,177,211,155,172,211,...,181,137,145,189,147,166,192,157,185,169,129,144,148,114,122,157,125,137,157,127,147,154,128,147,130,102,114,87,63,73,44,26,36,25,12,17,25,12,15,2
4,158,113,139,194,144,174,215,162,191,225,179,214,232,189,222,235,193,219,234,191,206,238,200,219,244,213,240,243,213,239,242,210,238,244,216,241,247,224,244,246,...,238,195,209,237,191,200,238,194,201,239,197,216,238,195,219,232,179,201,217,150,173,224,171,207,227,177,206,222,178,201,209,166,185,172,135,149,109,78,92,2


# Data Generator

In [0]:
from PIL import Image
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.python.keras.utils import Sequence
import random

def get_images(path_list, image_size = 224):
  n = len(path_list)
  images = np.zeros((n,image_size,image_size,3))
  for i in range(n):
    images[i] = np.array(Image.open(path_list[i]).resize((image_size,image_size)))
  return images

def to_one_hot(labels):
  return np.eye(7)[labels]


class Stratified_Batch_Generator(Sequence):
  def __init__(self,image_size,paths,batch_size,augments):
    self.image_size = image_size
    self.paths = paths
    self.batch_size = batch_size
    self.augments = augments

  def __len__(self):
    return 10015

  def __getitem__(self,idx):
    batch_paths = []
    labels = []
    for k in range(len(self.paths)):
      sub_paths = self.paths[k]
      for i in range(int(self.batch_size / len(self.paths))):
        batch_paths.append(sub_paths[random.randint(0,len(sub_paths)- 1)])
        labels.append(k)
    batch = get_images(batch_paths, self.image_size)
    labels = to_one_hot(labels)
    if(self.augments):
      return self.augments.flow(batch,labels, batch_size = self.batch_size).next()
    else:
      return (batch,labels)




In [0]:

import os
meta_data = pd.read_csv('Data/skin-cancer-mnist-ham10000/HAM10000_metadata.csv')
reverse_label_map = {
    'bkl' : 0,
    'nv'  : 1,
    'df'  : 2, 
    'mel' : 3,
    'vasc': 4,
    'bcc' : 5,
    'akiec': 6
}
label_map = ['bkl','nv', 'df', 'mel', 'vasc', 'bcc', 'akiec']
paths = ['Data/skin-cancer-mnist-ham10000/HAM10000_images_part_1/','Data/skin-cancer-mnist-ham10000/HAM10000_images_part_2/']
def get_path_list(df):
  path_list = []
  size = len(meta_data['dx'].unique())
  for i in range(size):
    path_list.append([])
  for path in paths:
    file_list = os.listdir(path)
    for file_name in file_list:
      path_list[reverse_label_map[meta_data.loc[meta_data['image_id'] == file_name[:-4]]['dx'].values[0]]].append(path + file_name)
  return path_list

def split_train_test(path_list, percentage):
  train_path_list = []
  test_path_list = []
  for i in range(len(path_list)):
    train_path_list.append(path_list[i][:int(len(path_list[i]) * percentage)])
    test_path_list.append(path_list[i][int(len(path_list[i]) * percentage):])
  return (train_path_list, test_path_list)

path_list = get_path_list(meta_data)
train_path_list,test_path_list = split_train_test(path_list,0.8)

In [0]:
len(test_path_list[1])

1341

In [0]:
train_augment = ImageDataGenerator(
    rotation_range = 180,
    width_shift_range=0.1,
    height_shift_range=0.1,
    rescale = 1./255,
    horizontal_flip=True,
    brightness_range = [0.2,0.8],
    fill_mode='nearest',
    zoom_range= [0.9,1.1],
)

test_augment = ImageDataGenerator(
    rescale = 1./255,
    horizontal_flip=True,
    rotation_range = 180,
    fill_mode='nearest'
)

In [0]:
train_generator = Stratified_Batch_Generator(64, train_path_list, 70,train_augment)
test_generator = Stratified_Batch_Generator(64, test_path_list, 70,test_augment)

## Model

In [25]:
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.layers import Input,GlobalAveragePooling2D, Conv2D, Dense,Dropout,Flatten,Activation,BatchNormalization,MaxPooling2D
from tensorflow.keras.optimizers import Adam,Nadam
from tensorflow.keras.models import Model

def Small_Alexnet(input_shape, num_class, weights = None):
  inputs = Input(input_shape)
  conv1 = Conv2D(32,kernel_size=(7,7),strides=2,padding="same")(inputs)
  relu1 = Activation('relu')(conv1)
  pool1 = MaxPooling2D((3,3),strides=2)(relu1)
  bn1 = BatchNormalization()(pool1)

  conv2 = Conv2D(64,kernel_size=(5,5),strides=1,padding="same")(bn1)
  relu2 = Activation('relu')(conv2)
  pool2 = MaxPooling2D((3,3),strides=2)(relu2)
  bn2 = BatchNormalization()(pool2)

  conv3 = Conv2D(64,kernel_size=(3,3),strides=1,padding="same")(bn2)
  relu3 = Activation('relu')(conv3)
  bn3 = BatchNormalization()(relu3)

  conv3 = Conv2D(64,kernel_size=(3,3),strides=1,padding="same")(bn3)
  relu3 = Activation('relu')(conv3)
  bn3 = BatchNormalization()(relu3)

  conv3 = Conv2D(64,kernel_size=(5,5),strides=1,padding="same")(bn3)
  relu3 = Activation('relu')(conv3)
  bn3 = BatchNormalization()(relu3)

  conv3 = Conv2D(64,kernel_size=(5,5),strides=1,padding="same")(bn3)
  relu3 = Activation('relu')(conv3)
  bn3 = BatchNormalization()(relu3)
  pool3 = MaxPooling2D((3,3),strides=2)(bn3)

  flat = Flatten()(pool3)
                       
  model=Model(inputs=inputs,outputs=outputs)
  model.compile(loss='categorical_crossentropy',
	              optimizer=Nadam(),
	              metrics=['accuracy'])
  if(weights != None):
    model.load_weights(weights)
  return model

model = Small_Alexnet((64,64,3),7,None)
model.summary()

Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 64, 64, 3)]       0         
_________________________________________________________________
conv2d_54 (Conv2D)           (None, 32, 32, 32)        4736      
_________________________________________________________________
activation_54 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_27 (MaxPooling (None, 15, 15, 32)        0         
_________________________________________________________________
batch_normalization_54 (Batc (None, 15, 15, 32)        128       
_________________________________________________________________
conv2d_55 (Conv2D)           (None, 15, 15, 64)        51264     
_________________________________________________________________
activation_55 (Activation)   (None, 15, 15, 64)        0   

In [0]:
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
filepath="64-{epoch:02d}-{val_loss:.6f}.h5"
checkpoint = ModelCheckpoint('Model/' + filepath, monitor='val_loss', verbose=0, save_best_only=False, save_weights_only = True, mode='auto')
callbacks_list = [checkpoint] 
model.fit_generator(generator = train_generator,steps_per_epoch=1000,epochs=50,verbose=1,validation_data=test_generator,validation_steps=100,use_multiprocessing=True,workers=16,callbacks=callbacks_list)

Epoch 1/50
1000/1000 [==============================] - 1255s 1s/step - loss: 1.0891 - accuracy: 0.5817 - val_loss: 1.4043 - val_accuracy: 0.4997
Epoch 2/50
1000/1000 [==============================] - 1180s 1s/step - loss: 0.7985 - accuracy: 0.6909 - val_loss: 1.1967 - val_accuracy: 0.6073
Epoch 3/50
 937/1000 [===========================>..] - ETA: 1:10 - loss: 0.6707 - accuracy: 0.7367

In [0]:
Mmodel = MobileNetV2(input_shape=(224,224,3), weights='imagenet',include_top=True)
# Mmodel.summary()
len(Mmodel.layers)

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_11[0][0]                   
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
_______________________________________________________________________________

157

In [0]:
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense,Dropout,Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model

def build():
  base_model=MobileNetV2(input_shape=(224,224,3), weights='imagenet',include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.
  base_model.trainable = False
  for layer in base_model.layers[-30:]:
    layer.trainable = True
  x=base_model.output
  x=GlobalAveragePooling2D()(x)
  # x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
  preds=Dense(7,activation='softmax')(x)
  model=Model(inputs=base_model.input,outputs=preds)
  model.compile(loss='categorical_crossentropy',
	              optimizer=Adam(lr=3e-4),
	              metrics=['accuracy'])
  return model
model = build()
model.summary()

Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_12[0][0]                   
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
____________________________________________________________________________________________

In [0]:
model.fit_generator(generator = train_generator,steps_per_epoch=100,epochs=50,verbose=1,validation_data=test_generator,validation_steps=10,use_multiprocessing=True,workers=4)

Epoch 1/50
